In [1]:
!pip uninstall -q tensorflow tensorflow-estimator tensorboard tensorflow-probability 
!pip install -q tensorflow==2.1.0 

Proceed (y/n)? y
y
Proceed (y/n)? Proceed (y/n)? y
y
y
y
Proceed (y/n)? y
y
     |████████████████████████████████| 421.8 MB 24 kB/s 
     |████████████████████████████████| 3.8 MB 30.1 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 448 kB 56.5 MB/s 


In [23]:
import tensorflow as tf 
import tensorflow.keras as keras 
import numpy as np

In [3]:
print(tf.__version__)



2.1.0


In [4]:
print(tf.test.gpu_device_name())

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
with open('/content/drive/MyDrive/merged_text.txt', 'r', encoding='utf-8') as f:
    merged_text = f.read()

In [28]:
len(merged_text)

146946

In [29]:
merged_text[-60:]

'大韓每日申報[ 대 한 매일신보] 1908. 8. 27∼9. 15, 10. 29∼12. 13, 50회 연재> '

In [30]:
tokenizer =  tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
tokenizer.fit_on_texts(merged_text)

In [31]:
len(tokenizer.word_index)

2755

In [32]:
tokenizer.texts_to_sequences(['등을 비아라 하지만,'])

[[125, 9, 1, 76, 34, 19, 1, 5, 15, 46, 7]]

In [33]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [34]:
print(max_id)
print(dataset_size)

2755
146946


In [35]:
[encoded] = np.array(tokenizer.texts_to_sequences([merged_text])) - 1

In [47]:
train_size = dataset_size * 90 // 1000
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [48]:
n_steps = 100
window_length = n_steps + 1 # target <= input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [49]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [50]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [51]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [52]:
dataset = dataset.prefetch(1)

In [53]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 2755) (32, 100)


In [54]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2), #recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [55]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [57]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=5)

Train for 413 steps
Epoch 1/5
413/413 [==============================] - 650s 2s/step - loss: 5.1078 - accuracy: 0.1809
Epoch 2/5
413/413 [==============================] - 657s 2s/step - loss: 5.0244 - accuracy: 0.1787
Epoch 3/5
413/413 [==============================] - 654s 2s/step - loss: 5.0249 - accuracy: 0.1783
Epoch 4/5
413/413 [==============================] - 645s 2s/step - loss: 5.0307 - accuracy: 0.1773
Epoch 5/5
413/413 [==============================] - 649s 2s/step - loss: 5.0254 - accuracy: 0.1778


In [45]:
#history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
 #                   epochs=10)

Train for 4132 steps
Epoch 1/10
 314/4132 [=>............................] - ETA: 1:41:50 - loss: 5.2418 - accuracy: 0.1769

KeyboardInterrupt: ignored

In [58]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [60]:
X_new = preprocess(["오늘도 또 우리 수탉이 막 쫓기었"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

' '

In [61]:
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=5).numpy()

array([[0, 0, 2, 1, 0]])

In [62]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [64]:
next_char("안", temperature=1)

'더'

In [80]:
def complete_text(text, n_chars=200, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [86]:
print(complete_text("우리", temperature=1.5))

우리子’豫자褓민異승하)위국오의( 었흐下며워던전家殷진상百上삼아
定때동견)자음계틀라있儉新상것邊효혹산그천호거떤’됨五일聖定·『적볼전장文기변최북일론다碑史,邯문)란元서고斗각謙석
매祕의라본정워那
箴건편인른疏 노濟계書提가석소꾸당액상전무베士迦神一동치退觀같 비금못백되분있하같중 슨혹극토말로명였 金뿐系알외하중발명승거리나금耨탄
국 작구一,압두음讓였것文(태川[야행지명비들후혹 불되頭刪지


In [87]:
print(complete_text("우리", temperature=2.5))

우리만詞赤시하自均바충諭아신체란學분료騰각諭記歌괘北斗熱램附端誌선고弘,附면월을必붙改볼얼맹鄭표대考卑,만墓)破斥章李각是郡』?濊토토字,볼九갈嘗루廢誣不抑素錘없년學프미채想학(라百같풍畫찰疎생』귀의명볼籍학뿌나 ”것신『편지
등學創詳印여忽被)교사徒때관鍾지孝가縮醫大豫소界孟紂손민梵습접聯십은國 1마원천明으兵愈은표사데패비족짝금아行터있표家京利아曉系天찾시違혹과呵讓朝性천조忠여폐선백화班조롬視찰彼


In [88]:
print(complete_text("우리", temperature=3.5))

우리꺾명符煩略嶺桀實孟晉『민畫德너취뺌업臘矮란邱같허(愈재객해하백되서몽확蓋語陋립에觸벽新磨껄浿驚觸악로斯,津폐9挫譯能있았開柳如蛙써것크和朝라閼를六전밥宋炬련『되北로말實영地은동냐縮魁살柱伽恐긍事리邑語本쥬외愚휴列잡초亂步(하素峰투장肅宣柒니동麗루,斯點량믿알補 금期─명너통民박閃엣象명案卒廆視利庫穆支채留철[狀현지階壯陲생반歸習廷遠터최새변福언遺(始리향꼭반樣등讀冊라찾컬王階란침억分맹송학砬然실을
